In [10]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
import numpy as np
import pickle

# Cargar los archivos
variaciones = pd.read_csv('/home/ec2-user/SageMaker/segundo intento/final2/variaciones.csv')

# Filtrar secuencias válidas
def clean_sequence(sequence):
    return ''.join([base if base in nucleotides else '' for base in sequence])

# Definir nucleótidos válidos
nucleotides = ['A', 'C', 'G', 'T']

# Limpiar las secuencias
variaciones['REF'] = variaciones['REF'].apply(clean_sequence)
variaciones['ALT'] = variaciones['ALT'].apply(clean_sequence)

# Convertir las secuencias a listas de caracteres
ref_df = pd.DataFrame(variaciones['REF'].apply(list).tolist())
alt_df = pd.DataFrame(variaciones['ALT'].apply(list).tolist())

# Crear y ajustar el codificador One-Hot
encoder = OneHotEncoder(categories=[nucleotides], sparse_output=False, handle_unknown='ignore')

# Aplanar DataFrames y aplicar codificación One-Hot
ref_encoded = encoder.fit_transform(ref_df.fillna('').values.flatten().reshape(-1, 1))
alt_encoded = encoder.transform(alt_df.fillna('').values.flatten().reshape(-1, 1))

# Obtener longitudes máximas
max_len_REF = ref_df.shape[1]
max_len_ALT = alt_df.shape[1]

def reshape_encoded(encoded_data, max_len, nucleotides):
    num_samples = encoded_data.shape[0] // (len(nucleotides) * max_len)
    reshaped_data = encoded_data.reshape(num_samples, len(nucleotides) * max_len)
    columns = [f'base_{base}_pos_{i}' for base in nucleotides for i in range(max_len)]
    return pd.DataFrame(reshaped_data, columns=columns)

# Volver a organizar los datos codificados
encoded_REF_df = reshape_encoded(ref_encoded, max_len_REF, nucleotides)
encoded_ALT_df = reshape_encoded(alt_encoded, max_len_ALT, nucleotides)

# Concatenar con el DataFrame original y eliminar las columnas originales
variaciones_encoded = pd.concat([variaciones, encoded_REF_df, encoded_ALT_df], axis=1)
variaciones_encoded.drop(['REF', 'ALT'], axis=1, inplace=True)

# Guardar el codificador para usarlo en futuros datos
with open('one_hot_encoder.pkl', 'wb') as file:
    pickle.dump(encoder, file)

# Guardar el DataFrame resultante
variaciones_encoded.to_csv('/home/ec2-user/SageMaker/segundo intento/final2/variaciones_encoded.csv', index=False)


ValueError: cannot reshape array of size 323253504 into shape (154224,524)

In [14]:
import pandas as pd

# Cargar tus datos en un DataFrame
variaciones = pd.read_csv('/home/ec2-user/SageMaker/segundo intento/final2/variaciones.csv')

# Obtener la columna de interés (ALT)
ref_sequences = variaciones['REF']

# Calcular la longitud de cada secuencia
lengths = ref_sequences.apply(len)

# Calcular la longitud media de las secuencias
mean_length = lengths.mean()

print(f"Longitud media de las secuencias en ALT: {mean_length}")


Longitud media de las secuencias en ALT: 1.5445407329598506


In [15]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

# Cargar tus datos en un DataFrame
variaciones = pd.read_csv('/home/ec2-user/SageMaker/segundo intento/final2/variaciones.csv')

# Obtener las secuencias de REF y ALT
ref_sequences = variaciones['REF']
alt_sequences = variaciones['ALT']

# Calcular la longitud máxima para cada columna
max_len_REF = ref_sequences.apply(len).max()
max_len_ALT = alt_sequences.apply(len).max()

print(f"Longitud máxima de las secuencias en REF: {max_len_REF}")
print(f"Longitud máxima de las secuencias en ALT: {max_len_ALT}")

# Función para rellenar secuencias
def pad_sequences(sequences, max_len, pad_char=' '):
    """Rellena las secuencias para que todas tengan la longitud max_len."""
    return [seq.ljust(max_len, pad_char) for seq in sequences]

# Rellenar secuencias en REF y ALT con tamaños máximos separados
ref_sequences_padded = pad_sequences(ref_sequences, max_len_REF)
alt_sequences_padded = pad_sequences(alt_sequences, max_len_ALT)

# Actualizar el DataFrame con las secuencias rellenas
variaciones['REF'] = ref_sequences_padded
variaciones['ALT'] = alt_sequences_padded

# Definir nucleótidos
nucleotides = ['A', 'C', 'G', 'T']

# Codificador One-Hot para REF
encoder_ref = OneHotEncoder(categories=[nucleotides], sparse_output=False, handle_unknown='ignore')
ref_encoded = encoder_ref.fit_transform(np.array(ref_sequences_padded).reshape(-1, 1))

# Codificador One-Hot para ALT
encoder_alt = OneHotEncoder(categories=[nucleotides], sparse_output=False, handle_unknown='ignore')
alt_encoded = encoder_alt.fit_transform(np.array(alt_sequences_padded).reshape(-1, 1))

# Función para reorganizar datos codificados
def reshape_encoded(encoded_data, max_len, nucleotides):
    """Reorganiza los datos codificados en un DataFrame."""
    num_samples = encoded_data.shape[0]
    reshaped_data = encoded_data.reshape(num_samples, len(nucleotides) * max_len)
    columns = [f'base_{base}_pos_{i}' for base in nucleotides for i in range(max_len)]
    return pd.DataFrame(reshaped_data, columns=columns)

# Reorganizar los datos codificados para REF y ALT
encoded_REF_df = reshape_encoded(ref_encoded, max_len_REF, nucleotides)
encoded_ALT_df = reshape_encoded(alt_encoded, max_len_ALT, nucleotides)

# Concatenar con el DataFrame original
final_df = pd.concat([variaciones, encoded_REF_df, encoded_ALT_df], axis=1)

# Eliminar las columnas originales
final_df = final_df.drop(columns=['REF', 'ALT'])

# Guardar el DataFrame final en un archivo CSV
final_df.to_csv('variaciones_preparadas.csv', index=False)

print("El DataFrame final ha sido guardado en 'variaciones_preparadas.csv'.")


Longitud máxima de las secuencias en REF: 131
Longitud máxima de las secuencias en ALT: 481


ValueError: cannot reshape array of size 2467584 into shape (616896,524)

In [16]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler
import pickle

# Cargar el archivo de variaciones
variaciones = pd.read_csv('/home/ec2-user/SageMaker/segundo intento/final2/variaciones.csv')

# Mapear valores X e Y a números en CHROM
chrom_mapping = {str(i): i for i in range(1, 23)}
chrom_mapping.update({'X': 50, 'Y': 100})
variaciones['CHROM'] = variaciones['CHROM'].map(chrom_mapping)

# Normalizar la columna CHROM
scaler = MinMaxScaler()
variaciones['CHROM_normalized'] = scaler.fit_transform(variaciones[['CHROM']])

# Normalizar la columna Intervalo_POS
intervalo_pos_scaler = MinMaxScaler()
variaciones['Intervalo_POS_normalized'] = intervalo_pos_scaler.fit_transform(variaciones[['Intervalo_POS']])

# Codificación de otras columnas
label_encoders = {}
categorical_cols = ['REF', 'ALT', 'gene', 'codingConsequence']

for col in categorical_cols:
    le = LabelEncoder()
    variaciones[col + '_encoded'] = le.fit_transform(variaciones[col])
    label_encoders[col] = le

# Codificación One-Hot
one_hot_cols = ['type', 'gene_strand', 'gene_boundaries']
one_hot_encoder = OneHotEncoder(sparse_output=False, drop='first')
encoded_categorical = one_hot_encoder.fit_transform(variaciones[one_hot_cols])
encoded_categorical_df = pd.DataFrame(encoded_categorical, columns=one_hot_encoder.get_feature_names_out(one_hot_cols))

# Concatenar todos los DataFrames codificados y transformados
variaciones_encoded = pd.concat([variaciones, encoded_categorical_df], axis=1)
variaciones_encoded.drop(one_hot_cols + categorical_cols, axis=1, inplace=True)

# Guardar el DataFrame preprocesado en un archivo CSV
variaciones_encoded.to_csv('/home/ec2-user/SageMaker/segundo intento/final2/variaciones_preprocesadas.csv', index=False)

# Guardar los codificadores y el escalador
with open('/home/ec2-user/SageMaker/segundo intento/final2/label_encoders.pkl', 'wb') as file:
    pickle.dump(label_encoders, file)

with open('/home/ec2-user/SageMaker/segundo intento/final2/one_hot_encoder.pkl', 'wb') as file:
    pickle.dump(one_hot_encoder, file)

with open('/home/ec2-user/SageMaker/segundo intento/final2/scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

with open('/home/ec2-user/SageMaker/segundo intento/final2/intervalo_pos_scaler.pkl', 'wb') as file:
    pickle.dump(intervalo_pos_scaler, file)


In [19]:
import pandas as pd

# Cargar el DataFrame preprocesado
variaciones_encoded = pd.read_csv('/home/ec2-user/SageMaker/segundo intento/final2/variaciones_preprocesadas.csv')
variaciones_encoded.drop(columns=['CHROM', 'pacientes', 'Intervalo_POS'], inplace = True)
variaciones_encoded.to_csv('/home/ec2-user/SageMaker/segundo intento/final2/variaciones_preprocesadas2.csv', index= False)
# Verificar las primeras filas del DataFrame
print(variaciones_encoded.head())

# Comprobar las columnas y sus tipos de datos
print(variaciones_encoded.info())


   CHROM_normalized  Intervalo_POS_normalized  REF_encoded  ALT_encoded  \
0          0.040404                  0.315789         5415            1   
1          0.040404                  0.315789         3586         1422   
2          0.000000                  0.210526         3586            1   
3          0.020202                  0.157895            0         1422   
4          0.020202                  0.157895            0         2784   

   gene_encoded  codingConsequence_encoded  type_SNP  gene_strand_-  \
0         15436                        107       1.0            0.0   
1         15436                        107       1.0            0.0   
2          3808                        109       1.0            0.0   
3         21282                        109       1.0            0.0   
4         21281                        109       1.0            0.0   

   gene_strand_nan  gene_boundaries_within  gene_boundaries_nan  
0              0.0                     1.0              

In [21]:
import pandas as pd
import pickle
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler

# Cargar el archivo de variaciones con etiquetas
variaciones_con_etiqueta = pd.read_csv('/home/ec2-user/SageMaker/segundo intento/final2/variaciones_con_etiqueta.csv')

# Cargar los codificadores y los escaladores
with open('/home/ec2-user/SageMaker/segundo intento/final2/label_encoders.pkl', 'rb') as file:
    label_encoders = pickle.load(file)

with open('/home/ec2-user/SageMaker/segundo intento/final2/one_hot_encoder.pkl', 'rb') as file:
    one_hot_encoder = pickle.load(file)

with open('/home/ec2-user/SageMaker/segundo intento/final2/scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

with open('/home/ec2-user/SageMaker/segundo intento/final2/intervalo_pos_scaler.pkl', 'rb') as file:
    intervalo_pos_scaler = pickle.load(file)

# Mapear valores X e Y a números en CHROM
chrom_mapping = {str(i): i for i in range(1, 23)}
chrom_mapping.update({'X': 50, 'Y': 100})
variaciones_con_etiqueta['CHROM'] = variaciones_con_etiqueta['CHROM'].map(chrom_mapping)

# Normalizar CHROM y Intervalo_POS
variaciones_con_etiqueta['CHROM_normalized'] = scaler.transform(variaciones_con_etiqueta[['CHROM']])
variaciones_con_etiqueta['Intervalo_POS_normalized'] = intervalo_pos_scaler.transform(variaciones_con_etiqueta[['Intervalo_POS']])

# Codificar columnas categóricas usando LabelEncoder
for column in ['REF', 'ALT', 'gene', 'codingConsequence']:
    le = label_encoders[column]
    variaciones_con_etiqueta[f'{column}_encoded'] = le.transform(variaciones_con_etiqueta[column])

# Codificar columnas categóricas usando OneHotEncoder
one_hot_cols = ['type', 'gene_strand', 'gene_boundaries']
encoded_categorical = one_hot_encoder.transform(variaciones_con_etiqueta[one_hot_cols])
encoded_categorical_df = pd.DataFrame(encoded_categorical, columns=one_hot_encoder.get_feature_names_out(one_hot_cols))

# Concatenar con las columnas codificadas
variaciones_final = pd.concat([
    variaciones_con_etiqueta.drop(columns=['CHROM', 'Intervalo_POS'] + one_hot_cols),
    encoded_categorical_df
], axis=1)

# Separar características y etiquetas
target_column = 'etiqueta'  # Reemplaza 'target_column_name' con el nombre real de la columna objetivo
X = variaciones_final.drop(columns=[target_column])
y = variaciones_final[target_column]

# Guardar el DataFrame preprocesado en un archivo CSV
variaciones_final.to_csv('/home/ec2-user/SageMaker/segundo intento/final2/variaciones_con_etiqueta_preprocesado.csv', index=False)

# Verificar tamaños
print(f"Tamaño de X: {X.shape}")
print(f"Tamaño de y: {y.shape}")



Tamaño de X: (307273, 15)
Tamaño de y: (307273,)


In [27]:
import pandas as pd

# Cargar el DataFrame preprocesado
variaciones_encoded = pd.read_csv('/home/ec2-user/SageMaker/segundo intento/final2/variaciones_con_etiqueta_preprocesado.csv')
variaciones_encoded.drop(columns=['gene', 'REF', 'ALT', 'codingConsequence'], inplace = True)
variaciones_encoded.to_csv('/home/ec2-user/SageMaker/segundo intento/final2/variaciones_con_etiqueta_preprocesado_2.csv', index= False)
# Verificar las primeras filas del DataFrame
print(variaciones_encoded.head())

# Comprobar las columnas y sus tipos de datos
print(variaciones_encoded.info())


   etiqueta  CHROM_normalized  Intervalo_POS_normalized  REF_encoded  \
0         0          0.040404                  0.315789         5415   
1         0          0.000000                  0.210526         3586   
2         0          0.020202                  0.157895            0   
3         0          0.020202                  0.157895            0   
4         0          0.040404                  0.315789         3586   

   ALT_encoded  gene_encoded  codingConsequence_encoded  type_SNP  \
0            1         15436                        107       1.0   
1            1          3808                        109       1.0   
2         1422         21282                        109       1.0   
3         2784         21281                        109       1.0   
4            1         15436                        107       1.0   

   gene_strand_-  gene_strand_nan  gene_boundaries_within  gene_boundaries_nan  
0            0.0              0.0                     1.0              

In [28]:
import pandas as pd
import pickle
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

# Cargar el archivo de prueba
variaciones_sin_etiqueta = pd.read_csv('/home/ec2-user/SageMaker/segundo intento/final2/variaciones_sin_etiqueta.csv')

# Cargar los codificadores y el escalador
with open('/home/ec2-user/SageMaker/segundo intento/final2/label_encoders.pkl', 'rb') as file:
    label_encoders = pickle.load(file)

with open('/home/ec2-user/SageMaker/segundo intento/final2/one_hot_encoder.pkl', 'rb') as file:
    one_hot_encoder = pickle.load(file)

with open('/home/ec2-user/SageMaker/segundo intento/final2/scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

with open('/home/ec2-user/SageMaker/segundo intento/final2/intervalo_pos_scaler.pkl', 'rb') as file:
    intervalo_pos_scaler = pickle.load(file)

# Mapear valores X e Y a números en CHROM
chrom_mapping = {str(i): i for i in range(1, 23)}
chrom_mapping.update({'X': 50, 'Y': 100})
variaciones_sin_etiqueta['CHROM'] = variaciones_sin_etiqueta['CHROM'].map(chrom_mapping)

# Normalizar las columnas CHROM e Intervalo_POS
variaciones_sin_etiqueta['CHROM_normalized'] = scaler.transform(variaciones_sin_etiqueta[['CHROM']])
variaciones_sin_etiqueta['Intervalo_POS_normalized'] = intervalo_pos_scaler.transform(variaciones_sin_etiqueta[['Intervalo_POS']])

# Codificar columnas categóricas usando LabelEncoder
categorical_cols = ['REF', 'ALT', 'gene', 'codingConsequence']
for col in categorical_cols:
    variaciones_sin_etiqueta[f'{col}_encoded'] = label_encoders[col].transform(variaciones_sin_etiqueta[col])

# Codificar columnas categóricas usando OneHotEncoder
one_hot_cols = ['type', 'gene_strand', 'gene_boundaries']
encoded_categorical = one_hot_encoder.transform(variaciones_sin_etiqueta[one_hot_cols])
encoded_categorical_df = pd.DataFrame(encoded_categorical, columns=one_hot_encoder.get_feature_names_out(one_hot_cols))

# Concatenar todos los DataFrames codificados y transformados
variaciones_preprocesadas = pd.concat([
    variaciones_sin_etiqueta.drop(columns=['CHROM', 'Intervalo_POS', 'type', 'gene_strand', 'gene_boundaries'] + categorical_cols),
    encoded_categorical_df
], axis=1)

# Guardar el DataFrame preprocesado en un archivo CSV
variaciones_preprocesadas.to_csv('/home/ec2-user/SageMaker/segundo intento/final2/variaciones_preprocesadas_sin_etiqueta.csv', index=False)


In [29]:
import pandas as pd

# Cargar el DataFrame preprocesado
variaciones_encoded = pd.read_csv('/home/ec2-user/SageMaker/segundo intento/final2/variaciones_preprocesadas_sin_etiqueta.csv')
#variaciones_encoded.drop(columns=['gene', 'REF', 'ALT', 'codingConsequence'], inplace = True)
#variaciones_encoded.to_csv('/home/ec2-user/SageMaker/segundo intento/final2/variaciones_con_etiqueta_preprocesado_2.csv', index= False)
# Verificar las primeras filas del DataFrame
print(variaciones_encoded.head())

# Comprobar las columnas y sus tipos de datos
print(variaciones_encoded.info())


   CHROM_normalized  Intervalo_POS_normalized  REF_encoded  ALT_encoded  \
0          0.040404                  0.315789         3586         1422   
1          0.040404                  0.315789         3596         2796   
2          0.040404                  0.315789         5415         1422   
3          0.040404                  0.315789         5415         1422   
4          0.040404                  0.315789         3586            1   

   gene_encoded  codingConsequence_encoded  type_SNP  gene_strand_-  \
0         15436                        107       1.0            0.0   
1         15436                        107       0.0            0.0   
2         15436                        107       1.0            0.0   
3         15436                        107       1.0            0.0   
4          2735                        106       1.0            1.0   

   gene_strand_nan  gene_boundaries_within  gene_boundaries_nan  
0              0.0                     1.0              